In [ ]:
import numpy as np
import pickle as pk
from tabulate import tabulate
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

test_file = open('test_dataset.pickle', 'rb')
test_data = pk.load(test_file)

train_file = open('train_dataset.pickle', 'rb')
train_data = pk.load(train_file)

train_data = np.random.permutation(train_data)
train_sets = np.split(train_data, 20)

tx = test_data[:,0].reshape((-1, 1))
ty = test_data[:,1]

mse = [0]*15
bias = [0]*15
error = [0]*15
degree = [0]*15
variance = [0]*15
            
for d in range(1, 16):
    degree[d-1] = d
    predicted = []
    for s in range(0, 20):
        x = train_sets[s][:,0].reshape((-1, 1))
        x_ = PolynomialFeatures(degree=d, include_bias=False).fit_transform(x)
        y = train_sets[s][:,1]

        model = LinearRegression().fit(x_, y)
        tx_ = PolynomialFeatures(degree=d, include_bias=False).fit_transform(tx)
        y_pred = model.predict(tx_)
        predicted.append(y_pred)
        
    for p in range(0, 200):
        by = sum([item[p] for item in predicted]) / 20
        vy = sum(map(lambda i: i*i, [item[p] for item in predicted])) / 20
        bias_p = abs(by - ty[p])
        var_p = vy - by**2
        mse_p = ty[p]**2 + vy - 2*ty[p]*by
        bias[d-1] += bias_p
        variance[d-1] += var_p
        mse[d-1] += mse_p
        error[d-1] += mse_p - bias_p**2 - var_p
    mse[d-1] /= 200
    bias[d-1] /= 200
    error[d-1] /=  200
    variance[d-1] /= 200
    
plt.xlabel("Degree of Polynomial")
plt.ylabel("values")
plt.plot(degree[:10], [i**2 for i in bias[:10]], 'b', label = "Bias^2")
plt.plot(degree[:10], variance[:10], 'r', label = "Variance")
plt.plot(degree[:10], mse[:10], 'g', label = "MSE")
plt.show()

error = np.array(error).reshape((-1, 1))
print(tabulate(list(zip(degree, bias, variance, error)), headers=['degree', 'Bias', 'Variance', 'Irreducible Error'], tablefmt="grid"))